In [13]:
%%writefile README.md
# The Investigation Kit (TIK) — Orwell-style Minimal MVP

## What this is
- **Frontend:** Single-Page App (vanilla HTML/JS/CSS), no build.
- **Backend:** FastAPI (serve API + static files).
- **State:** In-memory store (seeded). Replace with DB later.

## Key UX (Orwell-like)
- Tabs (Reader/Listener/Insider/Profiler/Objectives/Log)
- Split layout (Left: Profile+Graph, Right: Source viewer)
- Highlighted *datachunks* in source → drag to Profile fields
- Conflict resolver (choose value)
- Objectives modal & Advisor overlay

## Run
```bash
make setup
make api
# open http://127.0.0.1:8000


Overwriting README.md


In [ ]:
%%writefile requirements.txt
fastapi==0.115.0
uvicorn[standard]==0.30.6
pydantic==2.8.2
python-multipart==0.0.9




Overwriting requirements.txt


In [15]:

%%writefile backend/main.py
from fastapi import FastAPI, HTTPException
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
from typing import Dict, List, Any

# ---------------- In-memory store (seed) ----------------
class Entity(BaseModel):
    id: str
    name: str | None = None
    avatar: str
    fields: Dict[str, str | None]
    evidence: Dict[str, List[Dict[str, Any]]]  # field -> [{value, sourceId}]

class Source(BaseModel):
    id: str
    kind: str  # reader|listener|insider
    title: str
    html: str  # already highlighted with <span class="chunk" ...>

class Store(BaseModel):
    entities: Dict[str, Entity]
    sources: Dict[str, Source]

def seed_store() -> Store:
    src = Source(
        id="src-001",
        kind="reader",
        title="City Gazette",
        html=(
            '<h2>City Gazette • Breaking</h2>'
            '<p>Authorities are investigating '
            '<span class="chunk" data-field="name" data-value="Evelyn Hart">Evelyn Hart</span> '
            '(also known as <span class="chunk" data-field="alias" data-value="Eva">Eva</span>), '
            'living at <span class="chunk" data-field="address" data-value="22 Palm Street">22 Palm Street</span>, '
            'previously at <span class="chunk" data-field="address" data-value="14 Cedar Ave">14 Cedar Ave</span>. '
            'Born on <span class="chunk" data-field="dob" data-value="1992-04-12">Apr 12, 1992</span>. '
            'Works at <span class="chunk" data-field="occupation" data-value="Aperture Analytics">Aperture Analytics</span>. '
            'Email <span class="chunk" data-field="email" data-value="evelyn.hart@aperture.ai">evelyn.hart@aperture.ai</span>, '
            'Phone <span class="chunk" data-field="phone" data-value="+1-202-555-0138">+1-202-555-0138</span>. '
            'Contacts: <span class="chunk" data-field="contact" data-value="Jonas Wilder">Jonas Wilder</span>, '
            '<span class="chunk" data-field="contact" data-value="Mira Koh">Mira Koh</span>. '
            'Handle <span class="chunk" data-field="account" data-value="@eva_h">@eva_h</span>.</p>'
        ),
    )
    ent = Entity(
        id="ent-001",
        name=None,
        avatar="https://api.dicebear.com/9.x/shapes/svg?seed=evelyn",
        fields={
            "name": None, "alias": None, "dob": None, "address": None,
            "occupation": None, "email": None, "phone": None, "account": None, "contact": None
        },
        evidence={}
    )
    return Store(entities={ent.id: ent}, sources={src.id: src})

STORE = seed_store()

# ---------------- API models ----------------
class CommitChunkIn(BaseModel):
    entityId: str
    field: str
    value: str
    sourceId: str

class ResolveIn(BaseModel):
    entityId: str
    field: str
    chosen: str

# ---------------- App ----------------
app = FastAPI(title="TIK Minimal API")

app.mount("/", StaticFiles(directory="frontend", html=True), name="static")

@app.get("/api/entities", response_model=List[Entity])
def list_entities():
    return list(STORE.entities.values())

@app.get("/api/sources", response_model=List[Source])
def list_sources(kind: str | None = None):
    srcs = list(STORE.sources.values())
    return [s for s in srcs if s.kind == kind] if kind else srcs

@app.post("/api/chunks/commit", response_model=Entity)
def commit_chunk(payload: CommitChunkIn):
    ent = STORE.entities.get(payload.entityId)
    if not ent:
        raise HTTPException(404, "Entity not found")
    ent.evidence.setdefault(payload.field, []).append({"value": payload.value, "sourceId": payload.sourceId})
    # Set field only if empty; conflicts resolved by /resolve
    if ent.fields.get(payload.field) in (None, ""):
        ent.fields[payload.field] = payload.value
    STORE.entities[ent.id] = ent
    return ent

@app.post("/api/chunks/resolve", response_model=Entity)
def resolve_conflict(payload: ResolveIn):
    ent = STORE.entities.get(payload.entityId)
    if not ent:
        raise HTTPException(404, "Entity not found")
    ent.fields[payload.field] = payload.chosen
    STORE.entities[ent.id] = ent
    return ent

# health
@app.get("/api/health", response_class=HTMLResponse)
def health():
    return "ok"


Overwriting backend/main.py


In [16]:

%%writefile frontend/index.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <title>TIK • Orwell-style Minimal</title>
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <link href="/styles.css" rel="stylesheet" />
</head>
<body>
  <div id="app"></div>
  <script type="module" src="/app.js"></script>
</body>
</html>


Writing frontend/index.html


In [17]:

%%writefile frontend/styles.css
:root{
  --bg:#0b1220; --panel:#121a2b; --panel2:#0e1626; --accent:#0ea5e9;
  --text:#cbd5e1; --muted:#64748b; --border:#1f2a44; --warn:#f59e0b; --ok:#10b981;
}
*{box-sizing:border-box} body{margin:0;background:radial-gradient(ellipse at top,#0a1020,#070b16);color:var(--text);font:14px/1.6 system-ui,Segoe UI,Roboto}
.btn{background:#1e293b;border:1px solid var(--border);color:#e2e8f0;border-radius:10px;padding:.45rem .7rem;cursor:pointer}
.btn:hover{background:#243042}
.topbar{display:flex;justify-content:space-between;align-items:center;padding:.5rem 1rem;background:#0e1626;border-bottom:1px solid var(--border)}
.tabs button{margin-left:.25rem}
.grid{display:grid;grid-template-columns:5fr 7fr;height:calc(100vh - 48px)}
.left{border-right:1px solid var(--border);display:flex;flex-direction:column}
.section-h{padding:.4rem .75rem;border-bottom:1px solid var(--border);color:var(--muted);text-transform:uppercase;font-size:12px}
.profile{padding:10px}
.card{background:var(--panel);border:1px solid var(--border);border-radius:14px;padding:10px}
.fields{display:grid;grid-template-columns:1fr 1fr;gap:10px;margin-top:10px}
.field{border:1px solid var(--border);background:var(--panel2);padding:8px;border-radius:12px;min-height:64px}
.field.warn{border-color:#b45309;background:rgba(245,158,11,.08)}
.field .label{font-size:11px;color:var(--muted);text-transform:uppercase}
.field .val{margin-top:4px;min-height:20px;color:#e2e8f0}
.field .ev{margin-top:6px;font-size:10px;color:#74809a}
.right .content{padding:14px;overflow:auto;height:100%}
.chunk{background:rgba(253,224,71,.25);border-radius:6px;padding:0 4px}
.footer{position:absolute;bottom:6px;left:8px;font-size:12px;color:#94a3b8}
.modal{position:absolute;inset:0;display:flex;align-items:flex-start;justify-content:center}
.modal .backdrop{position:absolute;inset:0;background:rgba(0,0,0,.6)}
.modal .panel{position:relative;margin-top:60px;width:560px;background:#0e1626;border:1px solid var(--border);border-radius:16px;padding:16px}
.badge{display:inline-block;background:#172033;border:1px solid var(--border);border-radius:999px;padding:.1rem .5rem;font-size:12px;color:#cbd5e1}
.graph{border-top:1px solid var(--border);padding:10px}
.node{display:inline-block;padding:.25rem .6rem;border:1px solid var(--border);border-radius:999px;background:#172033;margin:4px}


Writing frontend/styles.css


In [18]:

%%writefile frontend/app.js
const el = (tag, cls, text) => {
  const n = document.createElement(tag);
  if (cls) n.className = cls;
  if (text) n.textContent = text;
  return n;
};

const state = {
  tabs: ["Reader","Listener","Insider","Profiler","Objectives","Log"],
  active: "Reader",
  entity: null,
  sources: [],
  lastDrag: null,
  showObjectives: false,
  advisor: null,
};

async function api(path, opts={}) {
  const res = await fetch(path, {headers:{'Content-Type':'application/json'}, ...opts});
  if (!res.ok) throw new Error(await res.text());
  return res.json();
}

async function loadInitial() {
  const [entities, sources] = await Promise.all([
    api("/api/entities"), api("/api/sources?kind=reader")
  ]);
  state.entity = entities[0];
  state.sources = sources;
}

function mountTopbar(root) {
  const bar = el("div","topbar");
  const left = el("div",null);
  left.append(el("div",null,"TIK • Investigation Workspace (Minimal)"));
  const tabs = el("div","tabs");
  state.tabs.forEach(t=>{
    const b = el("button","btn",t);
    b.onclick = () => {
      if (t==="Objectives") { state.showObjectives = true; render(); return; }
      state.active = t; render();
    };
    tabs.append(b);
  });
  const right = el("div",null,"09 Aug 2025 • 16:00");
  bar.append(left,tabs,right);
  root.append(bar);
}

function fieldBox(label, fieldKey) {
  const box = el("div","field");
  const lab = el("div","label",label);
  const val = el("div","val");
  const ev = el("div","ev");
  box.append(lab,val,ev);

  // drop
  box.ondragover = (e)=> e.preventDefault();
  box.ondrop = async (e)=>{
    e.preventDefault();
    const payload = e.dataTransfer.getData("application/json");
    if (!payload) return;
    const { field, value, sourceId } = JSON.parse(payload);
    if (field !== fieldKey) return;
    const ent = await api("/api/chunks/commit",{ method:"POST", body: JSON.stringify({
      entityId: state.entity.id, field, value, sourceId
    })});
    state.entity = ent;
    // advisor for key fields
    if (["name","dob"].includes(field)) state.advisor = `Good. You captured ${field.toUpperCase()}. Review Objectives.`;
    render();
  };

  // fill value & evidence & conflict UI
  const cur = state.entity.fields[fieldKey];
  val.textContent = cur || "— drop here —";
  const evidence = state.entity.evidence[fieldKey] || [];
  const uniq = [...new Set(evidence.map(e=>e.value))];
  if (uniq.length > 1) box.classList.add("warn");
  ev.textContent = `Evidence: ${evidence.length}`

  if (uniq.length > 1) {
    const wrap = el("div",null);
    uniq.forEach(v=>{
      const b = el("button","btn",`choose "${v}"`);
      b.style.marginRight = "6px";
      b.onclick = async ()=>{
        const ent = await api("/api/chunks/resolve",{ method:"POST", body: JSON.stringify({
          entityId: state.entity.id, field: fieldKey, chosen: v
        })});
        state.entity = ent; render();
      }
      wrap.append(b);
    });
    box.append(wrap);
  }

  return box;
}

function mountProfile(root) {
  const section = el("div","profile");
  const card = el("div","card");
  const top = el("div",null);
  const avatar = el("img"); avatar.src = state.entity.avatar; avatar.width=48; avatar.style.borderRadius="10px"; avatar.style.marginRight="10px";
  const name = el("span",null, state.entity.fields.name || "(unknown)");
  top.append(avatar, name);
  const fields = el("div","fields");
  const order = ["name","alias","dob","address","occupation","email","phone","account","contact"];
  order.forEach(k=>{
    fields.append(fieldBox(k, k));
  });
  card.append(top, fields);
  section.append(card);
  root.append(section);

  // mini graph stub
  const graph = el("div","graph");
  const title = el("div","label"); title.textContent="Graph (stub)";
  graph.append(title);
  const contacts = (state.entity.evidence["contact"] || []).map(e=>e.value);
  [...new Set(contacts)].slice(0,5).forEach(c=>{
    const n = el("div","node",c);
    graph.append(n);
  });
  root.append(graph);
}

function mountRight(root) {
  const head = el("div","section-h",state.active);
  const contentWrap = el("div","content");
  if (state.active === "Reader") {
    const s = state.sources[0];
    const cont = el("div");
    cont.innerHTML = s.html;
    // make chunks draggable
    cont.querySelectorAll(".chunk").forEach(span=>{
      span.setAttribute("draggable","true");
      span.addEventListener("dragstart",(e)=>{
        const field = span.getAttribute("data-field");
        const value = span.getAttribute("data-value");
        state.lastDrag = {field,value,sourceId: s.id};
        e.dataTransfer.setData("application/json", JSON.stringify(state.lastDrag));
      });
    });
    contentWrap.append(cont);
  } else {
    const stub = el("div",null,"This tab is stubbed in the minimal demo. Use Reader to drag highlighted chunks.");
    contentWrap.append(stub);
  }
  root.append(head, contentWrap);
}

function mountObjectivesModal(root){
  if (!state.showObjectives) return;
  const modal = el("div","modal");
  const backdrop = el("div","backdrop"); backdrop.onclick=()=>{state.showObjectives=false; render();}
  const panel = el("div","panel");
  const title = el("div",null); title.textContent = "Objectives";
  const ul = el("ul");
  const items = [
    { id:"obj1", text:"Xác định họ tên", check: !!state.entity.fields.name },
    { id:"obj2", text:"Ghi nhận DOB", check: !!state.entity.fields.dob },
    { id:"obj3", text:"Xác minh địa chỉ hiện tại", check: !!state.entity.fields.address },
  ];
  items.forEach(o=>{
    const li = el("li",null);
    const badge = el("span","badge", o.check ? "DONE" : "—");
    badge.style.marginRight="8px";
    li.append(badge, document.createTextNode(o.text));
    ul.append(li);
  });
  const actions = el("div",null);
  const close = el("button","btn","Close"); close.onclick=()=>{state.showObjectives=false; render();}
  actions.style.marginTop="10px"; actions.style.textAlign="right"; actions.append(close);
  panel.append(title,ul,actions);
  modal.append(backdrop,panel);
  root.append(modal);
}

function mountAdvisor(root){
  if (!state.advisor) return;
  const modal = el("div","modal");
  const backdrop = el("div","backdrop");
  const panel = el("div","panel");
  const title = el("div",null); title.textContent = "Advisor";
  const msg = el("div",null); msg.textContent = state.advisor;
  const btn = el("button","btn","Acknowledge");
  btn.style.marginTop="10px";
  btn.onclick=()=>{ state.advisor=null; render(); }
  panel.append(title,msg,btn);
  modal.append(backdrop,panel);
  root.append(modal);
}

function render() {
  const root = document.getElementById("app");
  root.innerHTML = "";
  mountTopbar(root);

  const grid = el("div","grid");
  const left = el("div","left");
  left.append(el("div","section-h","Profile / Graph / Timeline"));
  mountProfile(left);

  const right = el("div","right");
  mountRight(right);

  grid.append(left,right);
  root.append(grid);

  const footer = el("div","footer");
  footer.textContent = `Drag last: ${state.lastDrag ? `${state.lastDrag.field} → ${state.lastDrag.value}` : "—"}`;
  root.append(footer);

  mountObjectivesModal(root);
  mountAdvisor(root);
}

(async function bootstrap(){
  await loadInitial();
  render();
})();


Writing frontend/app.js
